# Quickstart : Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLFlow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform

In [6]:
# importing libraries
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature

In [2]:
# load dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=';')

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
# Split the data into train, validation and test split

train, test = train_test_split(data, test_size=0.25, random_state=17)

train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2330,7.5,0.38,0.29,12.7,0.050,25.0,209.0,0.9986,3.25,0.59,9.3,6
1194,6.2,0.37,0.22,8.3,0.025,36.0,216.0,0.9964,3.33,0.60,9.6,6
20,6.2,0.66,0.48,1.2,0.029,29.0,75.0,0.9892,3.33,0.39,12.8,8
1081,6.3,0.27,0.25,5.8,0.038,52.0,155.0,0.9950,3.28,0.38,9.4,6
1418,6.9,0.35,0.74,1.0,0.044,18.0,132.0,0.9920,3.13,0.55,10.2,5


In [5]:
train_x = train.drop(['quality'], axis=1)
train_y = train[['quality']].values.ravel() # transforming here into single dimension array by using ravel()

test_x = test.drop(['quality'], axis=1)
test_y = test['quality']

# Spliting train data into validation and train data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.25, random_state=17)

# Storing the signature

signature = infer_signature(train_x, train_y)

train_x.shape, train_y.shape, valid_x.shape, valid_y.shape


((2754, 11), (2754,), (919, 11), (919,))

In [ ]:
# ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])

    # compile the model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params["lr"],momentum=params["momentum"]),
                  loss='mean_squared_error',
                  metrics=[keras.metrics.RootMeanSquaredError()]
        )
    
    # Train the ANN Model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        
        model.fit(train_x, 
                  train_y,
                  validation_data=(valid_x, valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        # Log the params
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log the model
        